In [9]:
import sys
import hail as hl
import pandas as pd
import gnomad
from gnomad.resources.grch38 import gnomad
import pandas as pd
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

In [10]:
print("Load pre-filtered BGE data before annotating VEP")
mt = hl.read_matrix_table('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/01_Var_PreFilterQC.SCZ_BGE.mt') # from prefilter
print(mt.count()) # (4804848, 47683)

Load pre-filtered BGE data before annotating VEP


(155072, 70)


In [11]:
mt = hl.variant_qc(mt)

In [12]:
print("Low call rate filtering (>=0.95)")
print("Filtering out low call rate...")
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)
print(mt.count_rows()) # 4295392

Low call rate filtering (>=0.95)
Filtering out low call rate...


151824


In [13]:
# Filter out the invariant rows.
print("Remove monoallelic variants (invariants)")
mt = mt.filter_rows((mt.variant_qc.AF[0] > 0.0) & (mt.variant_qc.AF[0] < 1.0))
print(mt.count_rows())

Remove monoallelic variants (invariants)


151455


In [14]:
mt = mt.annotate_entries(DP=hl.sum(mt.AD))

In [15]:
print("pre filter variants")
mt = mt.filter_entries(
    hl.is_defined(mt.GT) &
    (
        (mt.GT.is_hom_ref() & 
            (
                (mt.GQ < 20) |
                (mt.DP < 10)
            )
        ) |
        (mt.GT.is_het() & 
            ( 
                ((mt.AD[1] / mt.DP) < 0.2) | 
                ((mt.AD[1] / mt.DP) > 0.8) | 
                (mt.GQ < 20) |
                (mt.DP < 10)
            )
        ) |
        (mt.GT.is_hom_var() & 
            (
                ((mt.AD[1] / mt.DP) < 0.8) |
                (mt.GQ < 20) |
                (mt.DP < 10)
            )
        )
    ),
    keep = False
)

pre filter variants


In [16]:
mt.count() # filter_entries -->  as missing --> equal num snp  

(151455, 70)

In [17]:
mt = hl.variant_qc(mt) # make DP using AD
mt_tmp = mt.filter_rows(mt.variant_qc.dp_stats.mean > 10)
mt_tmp.count_rows() # 4684298 

151390

In [19]:
print("Checkpointing... final variant QC by filter entries")
mt = mt.checkpoint('gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/03_filter_entries_VarQC.SCZ.mt', overwrite=True)

Checkpointing... final variant QC by filter entries


2025-07-16 14:14:02.211 Hail: INFO: wrote matrix table with 151455 rows and 70 columns in 24 partitions to gs://pbn_pilot/proj-PBN/04_rare_SNPs/00_raw_data/03_filter_entries_VarQC.SCZ.mt
